<a href="https://colab.research.google.com/github/pacificspatial/flateau/blob/main/notebook/sdsc_bootcamp_tokyo_202405.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FLATEAU (2D版PLATEAUデータ) ではじめる空間クエリ
by Hiroo Imaki (PSS) 2024/05/09

## はじめに
今年もSpatial Data Science Bootcampをやります！
国土交通省 Project PLATEAUの3Dデータをより扱いやすい2Dデータに変換した[Flateau](https://beta.source.coop/repositories/pacificspatial/flateau/description/) のデータとDuckDBを使ったハンズオンを昨年度のBootcampで行いました。今年のイベントでは違う内容を予定していますが、イベントを盛り上げるため、昨年度のSpatial Data Science Bootcamp で利用したこのノートを2023年度に更新された最新のPLATEAUのデータを利用できるようにアップデートしました！このノート上で「Google Colab」を利用して、カンタンに空間クエリを体験できますので、ぜひお試しください。
今年の会場ではもっと面白いことありますので、是非ご参加ください！会場でお会いできることを楽しみにしています。
[【2024年6月7日（金）Spatial Data Science Bootcamp Tokyo 2024 参加申込 受付中です！】](https://pacificspatial.com/carto/1472/)。

DuckDBは、2019年に開発が始まって、今年空間解析の機能拡張が追加された空間解析を行う人にとってはとても新しいツールです。
これまでは、PostgreSQLやGoogle BigQueryを使って空間データを解析することが多かったのですが、ローカル環境で大きな空間データを解析できるDuckDBは、空間解析を行う人は使えるようになっていたいツールの一つだと思います。

今回のチュートリアルは、SQLの基礎的な知識がある方で、空間データの解析に興味のある方が対象となっています。SQLについてよくわからないという方は、https://www.w3schools.com/sql/ をはじめたくさんのサイトがあるので、そちらを参照してください。また、前提として、Python が Notebook 環境で利用できる方を想定しています。ただし、なんの知識がなくても、このノート自体でチュートリアルが完結しているので、上から順番にセルを実行していただければ、手を動かしながら空間クエリについて学べるようになっています。学習に役立つサイトへのリンクも各所に記載しました。

皆さんが、空間クエリを活用して、さまざまな空間的な課題を解決するのに少しでもお役に立てば幸いです。

## 空間クエリとは
![SQLのイメージ](https://github.com/pacificspatial/flateau/blob/main/notebook/image/sql_screenshot.png?raw=1)

SQLとは、Structured Programming Language (シークゥオル、またはエスキューエルと読むことが多い）の略称で、リレーショナルデータベース(RDBMS)に格納されたデータを管理またデータに対し問い合わせするための言語です。文法は、データベースごとに（例えば、PostgreSQL/PostGIS、Bigqery SQL、Snowflake SQL, DuckDB）多少異なることがありますが、大きな違いはないため、 一度学ぶと、比較的簡単に他のデータベースでもクエリを書くことができます。

SQLを頻繁に使う人は、データエンジニア、データサイエンティスト、データ解析を行う人などです。

SQLを学ぶポイント：いわゆる　"the big 6" コマンドと呼ばれる、SELECT, FROM, WHERE, GROUP BY, HAVING, ORDER BY が文法の骨格です。SQLを書くときは、SQLコマンドは、できれば大文字で書くのが推奨されていますが、必須ではありません。空間クエリに関わるコマンドは、ST_で始まることが多く、代表的なものとしては、ST_BUFFER, ST_CENTERMEDIAN, ST_POLYGONIZE などがあります。実装されているコマンドの数は、データベースごと異なります。ちなみに、ST_は Spatial & Temporal の略です。

* [PostgreSQL/PostGIS](http://cse.naro.affrc.go.jp/yellow/pgisman/2.2.0/reference.html) 約260（ST関数以外も含む）
* [Google BigQuery](https://cloud.google.com/bigquery/docs/geospatial-data) 66
* [Snowflake](https://docs.snowflake.com/en/sql-reference/functions-geospatial) 66
* [DuckDB](https://duckdb.org/docs/extensions/spatial.html) 44

### なぜ空間クエリ？
今後ChatGPTなどの生成系AI技術発展により、データに対していろいろな問い合わせができるようになると思いますが、空間的な問い合わせの裏側では、おそらく空間クエリが走るようになると思います。例えば、「千代田区にあるコンビニって何軒あるの？」という問い合わせが一番効率的にできるのが、空間クエリです。例えば、クエリは以下のようになります。
空間SQLは、位置情報に関わる私たちが持つさまざまな問い合わせに対し答えを返してくれる問い合わせ言語で、問い合わせの文章が、空間クエリです。

```
SELECT COUNT (store.*)
FROM store, admin
WHERE admin = '千代田区'
AND ST_INTERSECTS (store.geom, admin.geom);
```

### 空間クエリを学べるリソース:
CARTOのブログには、空間クエリに関する詳しいチュートリアルがあります
 * [5 maps you didn't know you could create with SQL](https://carto.com/blog/create-maps-dataviz-with-sql)
 * [Ultimate Guide to Spatial Joins & Predicates with SQL](https://carto.com/blog/guide-to-spatial-joins-and-predicates-with-sql)
 * [空間クエリチートシート](https://drive.google.com/file/d/1ncaA_CES-lC4ZqzZ7iC__8kDpED9ieFr/view)

今日のプレゼンターのHelen McKenzie が既存のGISユーザ向けに[空間クエリの始め方についての記事](https://www.helenmakesmaps.com/post/how-to-sql-a-guide-for-gis-users)を書いています。

[University of Washington のサイト](https://uwescience.github.io/SQL-geospatial-tutorial/?utm_content=239999238&utm_medium=social&utm_source=linkedin&hss_channel=lcp-5084329)もチュートリアルや演習を通して空間SQLを学べるようになっています。

以上は、CARTOのSpatial Data Science Bootcampsのハンドブックからの抜粋たもので、一部このハンズオンのために内容を改変し日本語に翻訳しています。

また、以下がこのハンズオンの資料を作ったり勉強したりした時に使ったサイトや動画です。
* [Matt Forrest のYouTube動画](https://www.youtube.com/watch?v=ljzpm3Mrw-I&t=1426s)
* [Mark Litwintschik のブログ](https://tech.marksblogg.com/duckdb-gis-spatial-extension.html)
* [Mother Duck YouTube チャンネル](https://www.youtube.com/@motherduckdb)
* [DuckDB Spatial Github Repo](https://github.com/duckdblabs/duckdb_spatial)
* [DuckDB Spatial 機能拡張](https://duckdb.org/docs/extensions/spatial#spatial-copy-functions)
* [DuckDB Spatial ブログ](https://duckdb.org/2023/04/28/spatial.html)
* [Google BigQuery のGIS機能](https://cloud.google.com/bigquery/docs/geospatial-data)


## PLATEAUと都市のデジタルツインについて

[PLATEAU](https://www.mlit.go.jp/plateau/)とは、国土交通省が主導する、日本全国の3D都市モデルの整備・活用・オープンデータ化プロジェクトです。2024年時点で、合計221都市の3D都市モデルデータを[CityGML形式でオープンデータとして公開しています。](https://www.geospatial.jp/ckan/dataset/plateau)

[東京都デジタルツイン実現プロジェクト](https://info.tokyo-digitaltwin.metro.tokyo.lg.jp)は、都市のデジタルツインの実装を目指したプロジェクト。PLATEAUのデータも取り込みつつ、東京都の課題解決と都民のQOL向上を目指しています。

### 都市のデジタルツインと空間解析
都市のデジタルツインの一つの要は、空間データを活用し、市民への情報を提供したり、課題を解決することです。そのためには、空間データ解析がとても重要な位置を占めます。都市モデルが準拠するCityGML形式のデータは、もともと、さまざまな目的合わせ、変換して利用するためのデータフォーマットであるため、可視化のためには、例えば、3D Tiles形式に変換したり、解析のためには、空間解析に向いたデータ形式に変換して利用する必要があります。

* CityGMLの可視化 ---> 3D Tiles
* CityGMLの解析　 ---> GeoParquet, GeoPackage, Shapefile, etc

ただし、データの変換には、今のところ専門的な知識が必要とされるのが現状です。

### CityGMLの変換
なので、

[FME](https://www.safe.com)で[GeoParquet形式とGeoPackage形式に変換しました](https://beta.source.coop/repositories/pacificspatial/flateau/description/)。FMEを使ったPLATEAUデータ変換の概要は、[このサイトを参照してください](https://www.mlit.go.jp/plateau/learning/tpc04-1/)。

今後は変換した成果だけではなく、[FMEのワークスペースもGithubからアクセスできるようにしたい](https://github.com/pacificspatial/flateau/tree/main/workspace)と思っています。



##  ハンズオンの概要と準備
### ソフトウェアとデータ
ハンズオンでは、DuckDBを使って、GeoParquet形式に変換したPLATEAUの都市モデルデータを検索、解析します。
DuckDBを選んだのにはいくつか理由があります。
* インストールが簡単（通常のDBMSのようにサーバ・クライアントの構成ではない）
* 無茶苦茶早い
* 空間解析のための関数が最近実装
* GeoParquetが使える
* ネイティブPOINT_2D, BOX_2Dジオメトリ
* ベクタ化された実行モデルと列指向データ保存形式 ---> データを効率的クエリできる
* OLAP (online analytical processing)　---> 大量のデータに対して高速で多次元分析を行うソフトウェア。複数のリレーショナル・データ・セットからデータを抽出し、それを多次元形式に再編成することで、非常に高速な処理と非常に洞察に富んだ分析を実現
* インプロセスデータベースエンジン ---> クライアントーサーバ構成ではない。従来のサーバ経由のアクセス時間や通信時間のロスを減らせます．しかも，システム全体を統一的に管理でき，高可用性（システム・ダウンを起こさず，連続稼働する機能）を実現しやすい

DuckDBの位置付けに関するまとめは、[DuckDB入門](https://zenn.dev/notrogue/articles/1193d0ab8d8eda) にうまくまとめられていますので、参考にしてください。

GeoParquetを推すのは、いくつか理由があります
* Cloud Native フォーマットで、データサイズがとても小さくなる
* 最近QGISやFMEでも読み書きができるようになった
* Chris Holms、Javier de la Torre、Tim Shuab、Matt Forrest がいいと言っている

ただし、DuckDBは現時点では、以下の点で制限があります（対応予定ではある）。
* GiSTなどの空間インデックスに対応していない
* GEOMETRYタイプの仕様がフル実装されていない（ZやM、サブタイプなど）。
* GEOGRAPHYタイプに対応していないので、緯度経度を使った演算（面積計算など）ができない ([GEOMETRYとGEOGRAPHYについては後の方で説明しています。](#scrollTo=bredICAecbqK&line=6&uniqifier=1))
* PostGISに比べ利用できる関数がまだ少ない
* GeoParquetをテイティブでサポートしていない（[する予定](https://github.com/duckdblabs/duckdb_spatial/issues/27)）

# ハンズオン
### それでは実際にDuckDBを使う準備をします。

In [56]:
# 最初にハンズオンで使うライブラリをインストール
# データを地図として見るためのライブラリ
!pip install leafmap
!pip install pydeck

In [57]:
# install h3 extension
!wget https://pub-cc26a6fd5d8240078bd0c2e0623393a5.r2.dev/v0.10.2/linux_amd64_gcc4/h3ext.duckdb_extension.gz
!gunzip /content/h3ext.duckdb_extension.gz


--2024-05-10 00:53:43--  https://pub-cc26a6fd5d8240078bd0c2e0623393a5.r2.dev/v0.10.2/linux_amd64_gcc4/h3ext.duckdb_extension.gz
Resolving pub-cc26a6fd5d8240078bd0c2e0623393a5.r2.dev (pub-cc26a6fd5d8240078bd0c2e0623393a5.r2.dev)... 104.18.2.35, 104.18.3.35, 2606:4700::6812:323, ...
Connecting to pub-cc26a6fd5d8240078bd0c2e0623393a5.r2.dev (pub-cc26a6fd5d8240078bd0c2e0623393a5.r2.dev)|104.18.2.35|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9539436 (9.1M)
Saving to: ‘h3ext.duckdb_extension.gz.1’

h3ext.duckdb_extens 100%[===================>]   9.10M  --.-KB/s    in 0.1s    

2024-05-10 00:53:43 (63.2 MB/s) - ‘h3ext.duckdb_extension.gz.1’ saved [9539436/9539436]

gzip: /content/h3ext.duckdb_extension already exists; do you wish to overwrite (y or n)? ^C


In [59]:
# サンプルデータでleafmapが使えるか確認
import leafmap

m = leafmap.Map(center=[35.6812362,139.7645499], zoom=15)
in_geojson = 'https://flateau.s3.ap-northeast-1.amazonaws.com/data/plateau/sample/tokyo_station.geojson'
m.add_geojson(in_geojson, layer_name="PLATEAU SAMPLE")
m

Map(center=[35.6812362, 139.7645499], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_titl…

In [60]:
# ライブラリのインポート
# Duckdb is already in Colab!
import duckdb
import pandas as pd
import geopandas as gpd
from shapely import wkt

## DuckDBが動くかテスト
ここでは、「con」 というデータベース (test.db) への接続のためのオブジェクトを作成します。

In [92]:
# インメモリではなく、ファイルとしてデータを格納したいとき。この例では、test.dbというデータベースを作成する。
# データベース名、拡張子はなんでも良い。拡張子がなくても良い。
# 以下ではコネクションを作成しているが、デフォルトの設定で作成したもの、各種オプションを明示的に指定したもの、
# 外部データベースを指定したもの、インメモリを指定したもの、などの組み合わせで示した。

# デフォルトでの書き方
#con = duckdb.connect('test.db')

# オプションで、allow_unsigned_extensionsをtrueにすると、公式ではないH3拡張機能も読み込めるようになる。
#con = duckdb.connect(database="test.db", config={'allow_unsigned_extensions' : 'true'});


# in-memoryの場合は以下のように書く
#con = duckdb.connect(database=":memory:", read_only=False, config={"allow_unsigned_extensions": "true"});

# 接続を閉じる場合
#con.close()

# H3対応した読み込み方
con = duckdb.connect(config={"allow_unsigned_extensions": "true"})
con.load_extension('/content/h3ext.duckdb_extension')

In [97]:
# インメモリデータベースを使ってクエリが実行できるかテスト
con.sql('SELECT 42')

┌───────┐
│  42   │
│ int32 │
├───────┤
│    42 │
└───────┘

# DuckDBの基本操作

In [98]:
# リレーションの確認。クエリの結果を変数に格納して、他のクエリで呼び出せる。
r1 = con.sql('SELECT 42 AS i')
con.sql('SELECT i * 2 AS k FROM r1').show()

┌───────┐
│   k   │
│ int32 │
├───────┤
│    84 │
└───────┘



In [99]:
# インメモリにデータベースを作成。このノートを終了したらメモリから消える。
con.sql('DROP TABLE IF EXISTS ducks;')
con.sql("CREATE TABLE ducks AS SELECT 3 AS age, 'mandarin' AS breed;")

# DuckDBのスペシャルクエリ！
con.sql("FROM ducks;")

┌───────┬──────────┐
│  age  │  breed   │
│ int32 │ varchar  │
├───────┼──────────┤
│     3 │ mandarin │
└───────┴──────────┘

In [100]:
# テーブルを準備
con.sql('CREATE OR REPLACE TABLE junk(i INTEGER)')
con.sql('INSERT INTO junk VALUES (42)')
con.sql('INSERT INTO junk VALUES (21)')
# テーブルを表示
con.table('junk').show()

┌───────┐
│   i   │
│ int32 │
├───────┤
│    42 │
│    21 │
└───────┘



In [101]:
# 外部のファイルを読み込む。csv, parquet, json, Pandas DataFrame, Arrow objectに対応
# 読み込みのための関数を使うと以下のようにファイルを読み込める
#con.read_csv('sample_data/mnist_test.csv')
#con.read_parquet('example.parquet')
con.read_json('sample_data/anscombe.json')


┌─────────┬────────┬────────┐
│ Series  │   X    │   Y    │
│ varchar │ double │ double │
├─────────┼────────┼────────┤
│ I       │   10.0 │   8.04 │
│ I       │    8.0 │   6.95 │
│ I       │   13.0 │   7.58 │
│ I       │    9.0 │   8.81 │
│ I       │   11.0 │   8.33 │
│ I       │   14.0 │   9.96 │
│ I       │    6.0 │   7.24 │
│ I       │    4.0 │   4.26 │
│ I       │   12.0 │  10.84 │
│ I       │    7.0 │   4.81 │
│ ·       │     ·  │     ·  │
│ ·       │     ·  │     ·  │
│ ·       │     ·  │     ·  │
│ IV      │    8.0 │   5.76 │
│ IV      │    8.0 │   7.71 │
│ IV      │    8.0 │   8.84 │
│ IV      │    8.0 │   8.47 │
│ IV      │    8.0 │   7.04 │
│ IV      │    8.0 │   5.25 │
│ IV      │   19.0 │   12.5 │
│ IV      │    8.0 │   5.56 │
│ IV      │    8.0 │   7.91 │
│ IV      │    8.0 │   6.89 │
├─────────┴────────┴────────┤
│    44 rows (20 shown)     │
└───────────────────────────┘

In [102]:
# 外部のファイルを読み込む。SQL内に直接ファイルパスを書くこともできる
con.sql('SELECT * FROM "sample_data/anscombe.json"')

┌─────────┬────────┬────────┐
│ Series  │   X    │   Y    │
│ varchar │ double │ double │
├─────────┼────────┼────────┤
│ I       │   10.0 │   8.04 │
│ I       │    8.0 │   6.95 │
│ I       │   13.0 │   7.58 │
│ I       │    9.0 │   8.81 │
│ I       │   11.0 │   8.33 │
│ I       │   14.0 │   9.96 │
│ I       │    6.0 │   7.24 │
│ I       │    4.0 │   4.26 │
│ I       │   12.0 │  10.84 │
│ I       │    7.0 │   4.81 │
│ ·       │     ·  │     ·  │
│ ·       │     ·  │     ·  │
│ ·       │     ·  │     ·  │
│ IV      │    8.0 │   5.76 │
│ IV      │    8.0 │   7.71 │
│ IV      │    8.0 │   8.84 │
│ IV      │    8.0 │   8.47 │
│ IV      │    8.0 │   7.04 │
│ IV      │    8.0 │   5.25 │
│ IV      │   19.0 │   12.5 │
│ IV      │    8.0 │   5.56 │
│ IV      │    8.0 │   7.91 │
│ IV      │    8.0 │   6.89 │
├─────────┴────────┴────────┤
│    44 rows (20 shown)     │
└───────────────────────────┘

In colab, a multiple line edit/ a mass edit hortcut.
1. Alt + Click for Windows
2. Option + Click for MacOS

And to select all occurrences of the existing selection, use:

1. Ctrl + Shift + L for Windows
2. Command + Shift + L for MacOS

In [103]:
# 読み込んだデータをDataFrameなどに格納もできる
#con.sql('SELECT * FROM "sample_data/anscombe.json"').fetchall()   # Python objects
junk_df = con.sql('SELECT * FROM "sample_data/anscombe.json"').df()         # Pandas DataFrame
#con.sql('SELECT * FROM "sample_data/anscombe.json"').pl()         # Polars DataFrame
#con.sql('SELECT * FROM "sample_data/anscombe.json"').arrow()      # Arrow Table
#con.sql('SELECT * FROM "sample_data/anscombe.json"').fetchnumpy() # NumPy Arrays

junk_df.head()

,Series,X,Y
0,I,10.0,8.04
1,I,8.0,6.95
2,I,13.0,7.58
3,I,9.0,8.81
4,I,11.0,8.33


## Extension を確認

In [155]:
# 拡張機能の確認。デフォルトでは、httpfs, spatialなどは読み込まれていない。H3用のh3extが含まれているか確認。
con.sql('select * from duckdb_extensions()')

┌──────────────────┬─────────┬───────────┬───┬──────────────────────┬───────────────────┬───────────────────┐
│  extension_name  │ loaded  │ installed │ … │     description      │      aliases      │ extension_version │
│     varchar      │ boolean │  boolean  │   │       varchar        │     varchar[]     │      varchar      │
├──────────────────┼─────────┼───────────┼───┼──────────────────────┼───────────────────┼───────────────────┤
│ arrow            │ false   │ false     │ … │ A zero-copy data i…  │ []                │                   │
│ autocomplete     │ false   │ false     │ … │ Adds support for a…  │ []                │                   │
│ aws              │ false   │ false     │ … │ Provides features …  │ []                │                   │
│ azure            │ false   │ false     │ … │ Adds a filesystem …  │ []                │                   │
│ excel            │ false   │ false     │ … │ Adds support for E…  │ []                │                   │
│ fts     

## Extensionをインストールして読み込む

In [156]:
# HTTPS経由でファイルを読み込むための機能と、空間データ取扱のための機能を読み込む
con.sql("INSTALL 'httpfs'");
con.sql("INSTALL 'spatial'");

con.sql('LOAD httpfs')
con.sql('LOAD spatial')
con.sql('LOAD h3ext')

## 読み込める空間データファイルフォーマットを確認

In [106]:
# ドライバを確認。なんとGeoParquetはサポートされていない。でもParquetはサポートされているので大丈夫！
# そのうちサポートされるらしい。https://github.com/duckdblabs/duckdb_spatial/issues/27
con.sql('SELECT * FROM st_drivers()').df()

,short_name,long_name,can_create,can_copy,can_open,help_url
0,ESRI Shapefile,ESRI Shapefile,True,False,True,https://gdal.org/drivers/vector/shapefile.html
1,MapInfo File,MapInfo File,True,False,True,https://gdal.org/drivers/vector/mitab.html
2,UK .NTF,UK .NTF,False,False,True,https://gdal.org/drivers/vector/ntf.html
3,LVBAG,Kadaster LV BAG Extract 2.0,False,False,True,https://gdal.org/drivers/vector/lvbag.html
4,S57,IHO S-57 (ENC),True,False,True,https://gdal.org/drivers/vector/s57.html
5,DGN,Microstation DGN,True,False,True,https://gdal.org/drivers/vector/dgn.html
6,OGR_VRT,VRT - Virtual Datasource,False,False,True,https://gdal.org/drivers/vector/vrt.html
7,Memory,Memory,True,False,True,None
8,CSV,Comma Separated Value (.csv),True,False,True,https://gdal.org/drivers/vector/csv.html
9,GML,Geography Markup Language (GML),True,False,True,https://gdal.org/drivers/vector/gml.html


# ハンズオンのデータをDuckDBで取り扱う

## Geometryを取り扱えるか確認

In [107]:
con.sql('SELECT ST_POINT(0,0)')

┌────────────────┐
│ st_point(0, 0) │
│    geometry    │
├────────────────┤
│ POINT (0 0)    │
└────────────────┘

## 今回使うデータのリスト
* [建物データについて](https://beta.source.coop/repositories/pacificspatial/flateau/description/)
  * [建物セントロイド](https://beta.source.coop/pacificspatial/flateau/parquet/13101_chiyoda-ku_2023_building_centroid_lod0.parquet)
  * [建物フットプリントポリゴン](https://beta.source.coop/pacificspatial/flateau/parquet/13101_chiyoda-ku_2023_building_lod0.parquet)
  * [洪水浸水想定：属性テーブル](https://beta.source.coop/pacificspatial/flateau/parquet/13101_chiyoda-ku_2023_building_risk_flooding.parquet)
  * [高潮浸水想定：属性テーブル](https://beta.source.coop/pacificspatial/flateau/parquet/13101_chiyoda-ku_2023_building_risk_high_tide.parquet)
  * [地滑り：属性テーブル](https://beta.source.coop/pacificspatial/flateau/parquet/13101_chiyoda-ku_2023_building_risk_land_slide.parquet)
* [地形データについて](https://github.com/pacificspatial/flateau/blob/main/data/topography.md)
  * [標高 H3 Level10 ポリゴン](https://flateau.s3.ap-northeast-1.amazonaws.com/data/topography/tokyo23_elevation_h3lvl10.parquet)
  * [斜面傾斜 H3 Level10 ポリゴン](https://flateau.s3.ap-northeast-1.amazonaws.com/data/topography/tokyo23_slope_h3lvl10.parquet)


データを視覚化してみたいときは、上のリストのデータをダウンロードして、WindowsならQGISの最新版でGeoParquetがそのままで表示できます。

## ハンズオン用のデータを読み込む


*   GeoParquetは [ST_READ()](https://duckdb.org/docs/extensions/spatial#spatial-table-functions) ではそのまま読めないので、Parquetとして読む
*   読み込む際に、[ST_GeomFromWKB(geometry)](https://duckdb.org/docs/extensions/spatial#geometry-conversion) で blob で読み込まれるGeometryを変換する
*   Geometryを [DuckDBのネイティブジオメトリタイプ(POINT_2D等)](https://duckdb.org/docs/extensions/spatial.html) にキャストとして読み込むこともできる。ただし、[ネイティブタイプに対応した関数](https://github.com/duckdblabs/duckdb_spatial#supported-functions)はまだ限られている。




In [108]:
# 建物2Dポリゴンの読み込み。時間がかかると思う。httpfs 機能拡張を読み込んだので、インターネット経由でデータが読み込める
polygon_file = 'https://data.source.coop/pacificspatial/flateau/parquet/13101_chiyoda-ku_2023_building_lod0.parquet'

con.sql(
    "CREATE OR REPLACE TABLE building_polygon AS SELECT gml_id, cal_zmin_m, cal_area_m2, \
    cal_height_m::double cal_height_m, ST_GeomFromWKB(geom) AS geom FROM '{}' ".format(polygon_file)
    )

In [109]:
con.sql('SELECT * FROM building_polygon LIMIT 3')

┌──────────────────────┬────────────┬────────────────────┬──────────────┬──────────────────────────────────────────────┐
│        gml_id        │ cal_zmin_m │    cal_area_m2     │ cal_height_m │                     geom                     │
│       varchar        │   double   │       double       │    double    │                   geometry                   │
├──────────────────────┼────────────┼────────────────────┼──────────────┼──────────────────────────────────────────────┤
│ bldg_51a15704-7f1c…  │        6.8 │ 17.246979715497893 │        14.54 │ POLYGON ((139.7435658997431 35.67129960367…  │
│ bldg_4b1598c8-fe37…  │       7.36 │  769.3129709613212 │        33.51 │ POLYGON ((139.74251997288954 35.6720795705…  │
│ bldg_27afaf76-bb28…  │      28.37 │ 14.442584921285079 │          3.0 │ POLYGON ((139.73985888613043 35.6745975557…  │
└──────────────────────┴────────────┴────────────────────┴──────────────┴──────────────────────────────────────────────┘

In [110]:
# テーブルをDescribe
con.sql('DESCRIBE building_polygon')

┌──────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│ column_name  │ column_type │  null   │   key   │ default │  extra  │
│   varchar    │   varchar   │ varchar │ varchar │ varchar │ varchar │
├──────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ gml_id       │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ cal_zmin_m   │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ cal_area_m2  │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ cal_height_m │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ geom         │ GEOMETRY    │ YES     │ NULL    │ NULL    │ NULL    │
└──────────────┴─────────────┴─────────┴─────────┴─────────┴─────────┘

In [111]:
con.sql('SELECT count(*) FROM building_polygon')

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│        12293 │
└──────────────┘

In [112]:
con.sql('select cal_zmin_m, cal_area_m2, cal_height_m from building_polygon').df()

,cal_zmin_m,cal_area_m2,cal_height_m
0,6.800,17.246980,14.540
1,7.360,769.312971,33.510
2,28.370,14.442585,3.000
3,12.617,45.553370,8.423
4,7.400,943.510942,17.790
...,...,...,...
12288,2.570,36.273655,16.610
12289,2.430,115.141546,28.750
12290,2.550,32.405807,15.840
12291,1.930,1237.931097,35.710


In [113]:
# 標高データの読み込み（H3 level 10 のインデックスを含む）
elev_file = 'https://flateau.s3.ap-northeast-1.amazonaws.com/data/topography/tokyo23_elevation_h3lvl10.parquet'
con.sql("CREATE OR REPLACE TABLE elevation AS SELECT h3index10, val_max, val_mean, val_median, val_mode, ST_GeomFromWKB(geometry) AS geom FROM '{}'".format(elev_file))


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [114]:
# 斜面傾斜データの読み込み（H3 level 10 のインデックスを含む）
slope_file = 'https://flateau.s3.ap-northeast-1.amazonaws.com/data/topography/tokyo23_slope_h3lvl10.parquet'
con.sql("CREATE OR REPLACE TABLE slope AS SELECT h3index10, val_max, val_mean, val_median, val_mode, ST_GeomFromWKB(geometry) AS geom FROM '{}'".format(slope_file))


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [115]:
# 東京23区の標高分布
con.sql('SELECT val_max, val_mean, val_median, val_mode FROM elevation').df()

,val_max,val_mean,val_median,val_mode
0,6.19,2.178936,3.040,0.60
1,6.20,2.296221,3.205,3.36
2,6.18,2.424900,3.260,3.41
3,3.55,2.315523,2.250,2.25
4,6.00,3.568281,3.460,3.38
...,...,...,...,...
46727,7.53,3.395628,2.970,2.30
46728,7.60,5.528303,6.885,2.33
46729,12.43,7.627472,7.260,7.29
46730,6.92,2.909865,2.400,2.32


In [116]:
con.sql('SELECT val_max, val_mean, val_median, val_mode FROM slope').df()

,val_max,val_mean,val_median,val_mode
0,2.131054,0.610836,0.549421,0.298201
1,2.484312,0.704333,0.661217,0.654369
2,2.429445,0.844826,0.753571,0.469612
3,1.485541,0.495957,0.468135,0.075518
4,14.944903,1.953333,0.235748,0.111581
...,...,...,...,...
48651,1.653520,0.549164,0.524703,0.231044
48652,4.091684,1.030420,0.892122,0.572825
48653,NaN,NaN,NaN,NaN
48654,1.847704,0.770767,0.750116,0.405783


## 空間クエリ101


### 緯度経度、ジオメトリの取り扱いについて基本的なことを確認
#### ちょっとした用語や使う関数のメモ
* 緯度経度をジオメトリに変換する関数
 * ST_Point(float x_lon, float y_lat);
* 東京駅の緯度経度
 * latitude:35.6812362, longitude:139.7645499
* EPSG (European Petroleum Survey Group) コードについて、初めての人は以下を参照してください。
 * https://epsg.io/
 * https://epsg.org/home.html

### GEOMETRYタイプとGEOGRAPHYタイプ
空間データを取り扱う空間データベースでは、位置情報を格納するデータタイプとして、主に、GEOMETRYタイプか、GEOGRAPHYタイプを利用します。例えば、PostGISは、GEOMETRYタイプがデフォルトで、Google BigQueryはGeographyタイプがデフォルトです。今回利用するDuckDBは、GEOMETRYタイプのみが現在利用できますが、将来的には、GEOGRAPHYタイプも扱えるようになるようです。
これらGEOMETRYタイプとGEOGRAPHYタイプの違いは、簡単に言えば、GEOMETRYタイプは地表面を平らと仮定しているのに対し、GEOGRAPHYタイプは地球は球体に近いものだと仮定しています。なので、球体を仮定しているGEOGRAPHYタイプは、緯度経度を使ってジオメトリデータを保存し、GEOGRAPHYタイプに対応した関数を用いて、2地点間の距離を測れば、自動的に地球の丸さを考慮した長さを計算してくれます。その一方、GEOMETRYタイプでは、平らな空間を想定しているので、緯度経度の情報は一旦、平面上に投影する必要があります。

### DuckDBではどうすれば良いか？
DuckDBで距離や面積の計算をするときは、位置情報が緯度経度で格納されているときは、一旦、単位がメートルである、平面直角座標系やUTM座標系にジオメトリを変換してから計算を行う必要があります。

In [117]:
#緯度、経度 (Google)
con.sql("SELECT ST_POINT(35.6812362,139.7645499) geom")

┌────────────────────────────────┐
│              geom              │
│            geometry            │
├────────────────────────────────┤
│ POINT (35.6812362 139.7645499) │
└────────────────────────────────┘

In [118]:
#経度、緯度 (PostGIS: https://postgis.net/docs/ST_Point.html)
con.sql("SELECT ST_POINT(139.7645499, 35.6812362) geom")

┌────────────────────────────────┐
│              geom              │
│            geometry            │
├────────────────────────────────┤
│ POINT (139.7645499 35.6812362) │
└────────────────────────────────┘

In [119]:
# ST_POINT() では、本来、経度,緯度 の順番のはずなのだが、、、そうするとうまく動かないので、、、、
con.sql("SELECT ST_TRANSFORM(ST_POINT(35.6812362,139.7645499), 'EPSG:4326', 'EPSG:32654') geom")

┌─────────────────────────────────────────────┐
│                    geom                     │
│                  geometry                   │
├─────────────────────────────────────────────┤
│ POINT (388202.6519479657 3949296.928615559) │
└─────────────────────────────────────────────┘

In [120]:
# 経度・緯度のままだとうまくいかない
con.sql("SELECT ST_TRANSFORM(ST_POINT(139.7645499, 35.6812362), 'EPSG:4326', 'EPSG:32654') geom")

┌───────────────────────────┐
│           geom            │
│         geometry          │
├───────────────────────────┤
│ POINT (Infinity Infinity) │
└───────────────────────────┘

In [121]:
# 経度・緯度をフリップして緯度・経度にしてから計算するとうまくいく
con.sql("SELECT ST_TRANSFORM(ST_FlipCoordinates(ST_POINT(139.7645499,35.6812362)), 'EPSG:4326', 'EPSG:32654') geom")

┌─────────────────────────────────────────────┐
│                    geom                     │
│                  geometry                   │
├─────────────────────────────────────────────┤
│ POINT (388202.6519479657 3949296.928615559) │
└─────────────────────────────────────────────┘

In [122]:
# 東京と仙台の直線距離を計算。単位は、空間参照系の定義で決まります。この場合UTMなのでメートルが単位です。
con.sql("""
SELECT ST_DISTANCE(
  ST_TRANSFORM(ST_POINT(35.6812362,139.7645499), 'EPSG:4326', 'EPSG:32654'),
  ST_TRANSFORM(ST_POINT(38.3140913,140.4391724), 'EPSG:4326', 'EPSG:32654')
) distance
""")

┌──────────────────┐
│     distance     │
│      double      │
├──────────────────┤
│ 298197.928068306 │
└──────────────────┘

In [123]:
# ちなみに東京と仙台の直線距離を計算。緯度経度を投影しないで計算させると、、、気をつけましょう。
con.sql("""
SELECT ST_DISTANCE(
  ST_POINT(35.6812362,139.7645499),
  ST_POINT(38.3140913,140.4391724)
) distance
""")

┌───────────────────┐
│     distance      │
│      double       │
├───────────────────┤
│ 2.717911237531914 │
└───────────────────┘

In [124]:
# 東京駅からの直線距離を計算. XYを交換した後に計算。。。。
con.sql("select geom, ST_TRANSFORM(ST_FlipCoordinates(geom), 'EPSG:4326', 'EPSG:6677') from building_polygon limit 3")


┌──────────────────────┬───────────────────────────────────────────────────────────────────────────────────────────────┐
│         geom         │               st_transform(st_flipcoordinates(geom), 'EPSG:4326', 'EPSG:6677')                │
│       geometry       │                                           geometry                                            │
├──────────────────────┼───────────────────────────────────────────────────────────────────────────────────────────────┤
│ POLYGON ((139.7435…  │ POLYGON ((-36463.90800017322 -8126.404999974158, -36465.338000174015 -8120.661999970521, -3…  │
│ POLYGON ((139.7425…  │ POLYGON ((-36377.29000017048 -8221.0099999745, -36382.75000017285 -8227.9369999735, -36383.…  │
│ POLYGON ((139.7398…  │ POLYGON ((-36097.714000172986 -8461.642999974174, -36099.61400017492 -8461.840999973638, -3…  │
└──────────────────────┴───────────────────────────────────────────────────────────────────────────────────────────────┘

<h1>H3 空間インデックス</h1>

<a href="https://h3geo.org/docs/">H3</a> は、全世界を六角形で分割した形で位置情報をインデックス化するシステムです。H3は、Apache 2 のオープンソースライセンスで利用できます。巨大な地理空間データを効率よく扱うために活用できる空間インデックスです。

<a href="https://github.com/isaacbrodsky/h3-duckdb?tab=readme-ov-file">h3-duckdb</a> は、H3をDuckDBで使えるようにした機能拡張です。

In [126]:
# H3が使えるか確認
con.sql("SELECT h3_cell_to_latlng('822d57fffffffff');")

┌─────────────────────────────────────────┐
│  h3_cell_to_latlng('822d57fffffffff')   │
│                double[]                 │
├─────────────────────────────────────────┤
│ [44.104731208541175, 41.04892671698865] │
└─────────────────────────────────────────┘

In [127]:
# 緯度経度をインデックスに変換する
con.sql("select h3_latlng_to_cell(35.6812362,139.7645499, 15)")

┌────────────────────────────────────────────────┐
│ h3_latlng_to_cell(35.6812362, 139.7645499, 15) │
│                     uint64                     │
├────────────────────────────────────────────────┤
│                             644847777859667557 │
└────────────────────────────────────────────────┘

In [135]:
# 現在のテーブルにH3インデックスを付与する
# 格納列があるようなら落とす
con.sql('alter table building_polygon DROP COLUMN IF EXISTS h3index10')

# 格納列を作成
query_01 = '''
  alter table building_polygon ADD COLUMN IF NOT EXISTS h3index10 BIGINT;
'''
con.sql(query_01)

# H3インデックスを格納
query_01 = '''
  UPDATE building_polygon SET h3index10 = h3_latlng_to_cell(ST_Y(ST_CENTROID(geom)),ST_X(ST_CENTROID(geom)), 10);
'''
con.sql(query_01)

In [136]:
# 内容を確認
query_01 = '''
  SELECT * FROM building_polygon LIMIT 10;
'''
con.sql(query_01)

┌──────────────────────┬────────────┬────────────────────┬───┬──────────────────────┬────────────────────┐
│        gml_id        │ cal_zmin_m │    cal_area_m2     │ … │         geom         │     h3index10      │
│       varchar        │   double   │       double       │   │       geometry       │       int64        │
├──────────────────────┼────────────┼────────────────────┼───┼──────────────────────┼────────────────────┤
│ bldg_51a15704-7f1c…  │        6.8 │ 17.246979715497893 │ … │ POLYGON ((139.7435…  │ 622329812653473791 │
│ bldg_4b1598c8-fe37…  │       7.36 │  769.3129709613212 │ … │ POLYGON ((139.7425…  │ 622329812653572095 │
│ bldg_27afaf76-bb28…  │      28.37 │ 14.442584921285079 │ … │ POLYGON ((139.7398…  │ 622329812652523519 │
│ bldg_22575806-463c…  │     12.617 │  45.55336993050397 │ … │ POLYGON ((139.7464…  │ 622329812751253503 │
│ bldg_77fe66cf-eaa1…  │        7.4 │  943.5109417604941 │ … │ POLYGON ((139.7475…  │ 622329812751450111 │
│ bldg_1d4b3f41-6264…  │       7.39 │

### H3インデックスを使って空間クエリ
* DuckDB には、今の所ジオメトリに対して空間インデックスをPostGISのようには付与できないので、H3インデックスを明示的に使って空間結合します。

* 空間インデックスは、空間クエリでデータ解析するための最も重要な要素です。空間クエリでインデックスを利用できるかどうかで、実行速度に雲泥の差が出ます。

* [H3インデックス](https://h3geo.org/)は、Uber社が開発した、世界中を六角形に分割して、それぞれにユニークなIDを割り振るシステムで、ズームレベルにより、ごくごく小さい範囲から、日本全体を覆うようなレベルの範囲までさまざまなスケールで位置をインデックス化できます。

* 例えば、自分の座っている位置の緯度経度座標と隣に座っている人の座標に対し、あるレベル(解像度)のH3インデックスをつけておけば、二人が同じ場所にいるかどうかは、インデックス同士の比較だけで確認できます。もちろん、どのレベルを選択するかは、とても大事な問題ですが。

* H3には、Pythonなどで利用できるライブラリがあるので、H3インデックスを緯度経度から求めたり、レベルを変更したり、インデックスを活用するためのさまざまな関数が用意されています。

* DuckDBでも公式ではないですが、機能拡張としてH3を読み込むことができます ([リポジトリ](https://github.com/isaacbrodsky/h3-duckdb))。コンパイルをしてから読み込む必要があるため、ちょっと手間がかかるので、今回はインストールしませんが、ぜひ試してみてください。
 * [インストール方法の解説](https://tech.marksblogg.com/h3-duckdb-qgis.html)



In [137]:
# 建物が立っている地形の最大傾斜の大きい順に上位10件の建物のリストを取得
query_01 = '''
  SELECT t1.gml_id, t1.geom, t2.val_median
  FROM building_polygon t1, slope t2
  WHERE t1.h3index10 = t2.h3index10
  ORDER BY val_max DESC
  LIMIT 20
'''
con.sql(query_01)

┌──────────────────────┬──────────────────────────────────────────────────────────────────────────┬────────────────────┐
│        gml_id        │                                   geom                                   │     val_median     │
│       varchar        │                                 geometry                                 │       double       │
├──────────────────────┼──────────────────────────────────────────────────────────────────────────┼────────────────────┤
│ bldg_22c73fc7-b9e4…  │ POLYGON ((139.7354470095698 35.680677084621124, 139.73536902620373 35.…  │  2.653118133544922 │
│ bldg_b5deabbe-dd2f…  │ POLYGON ((139.73546120345992 35.6813575414582, 139.73543904892523 35.6…  │  2.653118133544922 │
│ bldg_a7fc9271-7d9f…  │ POLYGON ((139.73580890583366 35.68078896809503, 139.73580468226723 35.…  │  2.653118133544922 │
│ bldg_e10b086f-0dff…  │ POLYGON ((139.73456802769462 35.68150716082065, 139.73451287092604 35.…  │  2.653118133544922 │
│ bldg_8a985d7e-d7e6…  │ POLYGON

### 距離を使った空間クエリ

In [138]:
# 東京駅から全ての23区の建物への直線距離を計算
con.sql('''
  SELECT
    t1.gml_id,
    ST_DISTANCE(
      ST_TRANSFORM(ST_FlipCoordinates(geom), 'EPSG:4326', 'EPSG:6677'),
      ST_TRANSFORM(ST_POINT(35.6812362,139.7645499), 'EPSG:4326', 'EPSG:6677')
      ) distance
  FROM building_polygon t1
''')

┌───────────────────────────────────────────┬────────────────────┐
│                  gml_id                   │      distance      │
│                  varchar                  │       double       │
├───────────────────────────────────────────┼────────────────────┤
│ bldg_51a15704-7f1c-43f1-9d47-bf0024c6b126 │  2189.565443968769 │
│ bldg_4b1598c8-fe37-4ea2-a14a-481c02c87214 │ 2227.6474150441522 │
│ bldg_27afaf76-bb28-4431-b754-7d322d0481e3 │ 2351.2747994054243 │
│ bldg_22575806-463c-45c9-8419-693e97e6a7f9 │  1937.349226603361 │
│ bldg_77fe66cf-eaa1-43b2-ab48-0d1eb34b83da │  1825.611687798944 │
│ bldg_1d4b3f41-6264-4299-a3a1-9a8c4d01b66f │  2291.840415462184 │
│ bldg_5b678071-33ce-4ffd-9f38-58463cd3e25b │  2248.724560367671 │
│ bldg_6f70d203-933b-4dce-b86e-34a504b06679 │  1990.531338148698 │
│ bldg_f26a2b83-9ba9-47b2-9c45-03a56cdd87fe │ 1867.8273134211533 │
│ bldg_21a0d701-ea5c-4720-a9fe-d9e2ee2f1003 │  2311.924610128852 │
│                     ·                     │          ·      

In [139]:
# 東京駅から2km以内の建物の数、合計フットプリント面積、建物の高さの平均値を計算
# この場合、H3インデックスを使えないので、実行速度は遅い。H3ライブラリが使えれば最適化可能か。
%%time
con.sql('''
  SELECT
    count(t1.gml_id) cnt,
    sum(t1.cal_area_m2) tot_area,
    avg(cal_height_m) avg_height
  FROM building_polygon t1
  WHERE
    ST_DISTANCE(
      ST_TRANSFORM(ST_FlipCoordinates(ST_CENTROID(geom)), 'EPSG:4326', 'EPSG:6677'),
      ST_TRANSFORM(ST_POINT(35.6812362,139.7645499), 'EPSG:4326', 'EPSG:6677')
    ) <= 2000
''')

CPU times: user 10.1 ms, sys: 893 µs, total: 11 ms
Wall time: 14.2 ms


┌───────┬───────────────────┬───────────────────┐
│  cnt  │     tot_area      │    avg_height     │
│ int64 │      double       │      double       │
├───────┼───────────────────┼───────────────────┤
│  5878 │ 1579987.273729921 │ 18.70106766893501 │
└───────┴───────────────────┴───────────────────┘

In [140]:
# 同じ問題に対する別のアプローチ
con.sql('''
  SELECT
    count(t1.gml_id) cnt,
    sum(t1.cal_area_m2) tot_area,
    avg(cal_height_m) avg_height
  FROM building_polygon t1
  WHERE
    ST_INTERSECTS(
      ST_TRANSFORM(ST_FlipCoordinates(ST_CENTROID(geom)), 'EPSG:4326', 'EPSG:6677'),
      ST_BUFFER(ST_TRANSFORM(ST_POINT(35.6812362,139.7645499), 'EPSG:4326', 'EPSG:6677'), 2000)
    )
''')


┌───────┬────────────────────┬──────────────────┐
│  cnt  │      tot_area      │    avg_height    │
│ int64 │       double       │      double      │
├───────┼────────────────────┼──────────────────┤
│  5831 │ 1570777.4331581187 │ 18.7041625030012 │
└───────┴────────────────────┴──────────────────┘

### まずは建物データと斜面傾斜データを空間結合し、傾斜がきついところの上位1000建物を取得

In [141]:
query_02 = '''
  CREATE OR REPLACE TABLE high_slope_buildings AS
  SELECT t1.gml_id, t1.cal_area_m2, t1.cal_height_m, t1.geom, t2.val_max, t2.h3index10
  FROM building_polygon t1, slope t2
  WHERE t1.h3index10 = t2.h3index10
  ORDER BY val_max DESC
  LIMIT 1000
'''
con.sql(query_02)

### その上で、標高データと建物データを空間結合し、標高の低いところ順に並べ替え、上位100件の建物のリストを取得

In [142]:
query_03 = '''
SELECT t1.gml_id, t1.cal_area_m2, t1.cal_height_m, t1.val_max slope_max, t2.val_median elev_median, ST_AsText(t1.geom) geom
FROM high_slope_buildings t1, elevation t2
WHERE t1.h3index10 = t2.h3index10
ORDER BY elev_median ASC
LIMIT 100
'''
con.sql(query_03)

┌──────────────────────┬────────────────────┬────────────────────┬───┬────────────────────┬──────────────────────┐
│        gml_id        │    cal_area_m2     │    cal_height_m    │ … │    elev_median     │         geom         │
│       varchar        │       double       │       double       │   │       double       │       varchar        │
├──────────────────────┼────────────────────┼────────────────────┼───┼────────────────────┼──────────────────────┤
│ bldg_3ec997d2-a7db…  │ 132.28730667374006 │ 3.0600440000000004 │ … │  3.109999895095825 │ POLYGON ((139.7586…  │
│ bldg_c1bf1eb1-fb93…  │  19.24255708046654 │               7.71 │ … │               3.75 │ POLYGON ((139.7686…  │
│ bldg_06682e6e-1fb7…  │  3.232243699776628 │               3.01 │ … │               3.75 │ POLYGON ((139.7686…  │
│ bldg_a7b89248-7d33…  │  21.91952424741113 │               4.61 │ … │               3.75 │ POLYGON ((139.7696…  │
│ bldg_46eea36d-a677…  │ 14.520291885053922 │ 3.4080000000000013 │ … │          

### DuckDBのネイティブジオメトリタイプを試して見る

In [143]:
# DuckDBのネイティブ POINT_2D ジオメトリを追加してみる
con.sql('ALTER TABLE building_polygon ADD IF NOT EXISTS geom_2dp POINT_2D')
con.sql('update building_polygon set geom_2dp = ST_CENTROID(ST_FlipCoordinates(geom))::POINT_2D')

In [144]:
# 東京駅から全ての23区の建物への直線距離を計算
%%time
con.sql('''
  SELECT
    t1.gml_id,
    ST_DISTANCE(
      ST_TRANSFORM(geom_2dp, 'EPSG:4326', 'EPSG:6677'),
      ST_TRANSFORM(ST_POINT(35.6812362,139.7645499)::POINT_2D, 'EPSG:4326', 'EPSG:6677')
      ) distance
  FROM building_polygon t1
''')

CPU times: user 11.2 ms, sys: 906 µs, total: 12.1 ms
Wall time: 13.9 ms


┌───────────────────────────────────────────┬────────────────────┐
│                  gml_id                   │      distance      │
│                  varchar                  │       double       │
├───────────────────────────────────────────┼────────────────────┤
│ bldg_51a15704-7f1c-43f1-9d47-bf0024c6b126 │ 2192.5848910874597 │
│ bldg_4b1598c8-fe37-4ea2-a14a-481c02c87214 │    2241.9216155028 │
│ bldg_27afaf76-bb28-4431-b754-7d322d0481e3 │  2354.018664041131 │
│ bldg_22575806-463c-45c9-8419-693e97e6a7f9 │ 1944.4242768844258 │
│ bldg_77fe66cf-eaa1-43b2-ab48-0d1eb34b83da │  1849.483976014638 │
│ bldg_1d4b3f41-6264-4299-a3a1-9a8c4d01b66f │ 2304.8576248395316 │
│ bldg_5b678071-33ce-4ffd-9f38-58463cd3e25b │ 2260.4815084733173 │
│ bldg_6f70d203-933b-4dce-b86e-34a504b06679 │  1993.513571300217 │
│ bldg_f26a2b83-9ba9-47b2-9c45-03a56cdd87fe │ 1874.6741361509426 │
│ bldg_21a0d701-ea5c-4720-a9fe-d9e2ee2f1003 │ 2355.9163680475326 │
│                     ·                     │          ·      

In [145]:
# 東京駅から2km以内の建物の数、合計フットプリント面積、建物の高さの平均値を計算
%%time
con.sql('''
  SELECT
    count(t1.gml_id) cnt,
    sum(t1.cal_area_m2) tot_area,
    avg(cal_height_m) avg_height
  FROM building_polygon t1
  WHERE
    ST_DISTANCE(
      ST_TRANSFORM(geom_2dp, 'EPSG:4326', 'EPSG:6677'),
      ST_TRANSFORM(ST_POINT(35.6812362,139.7645499)::POINT_2D, 'EPSG:4326', 'EPSG:6677')
    ) < 2000
''')

CPU times: user 11.8 ms, sys: 0 ns, total: 11.8 ms
Wall time: 13.9 ms


┌───────┬───────────────────┬───────────────────┐
│  cnt  │     tot_area      │    avg_height     │
│ int64 │      double       │      double       │
├───────┼───────────────────┼───────────────────┤
│  5878 │ 1579987.273729921 │ 18.70106766893501 │
└───────┴───────────────────┴───────────────────┘

## クエリの結果をGeoJSONで出力
[COPYコマンド](https://duckdb.org/docs/extensions/spatial#spatial-copy-functions)を使うと、クエリの結果をファイルとして出力できる。

In [146]:
# GeoJSONとしてクエリの結果を保存
con.sql("copy ({}) to 'test.geojson' with (format gdal, driver 'GeoJSON', LAYER_CREATION_OPTIONS 'WRITE_BBOX=YES')".format(query_01))


## クエリの結果をDataFrameに格納

In [151]:
# change to DF
df = con.sql(query_03).df()
df

,gml_id,cal_area_m2,cal_height_m,slope_max,elev_median,geom
0,bldg_3ec997d2-a7db-4729-aeed-56515ba1ab03,132.287307,3.060044,29.956266,3.11,POLYGON ((139.75860220619106 35.68324645774352...
1,bldg_c1bf1eb1-fb93-47ff-8d9d-0aa9ebd5f9f0,19.242557,7.710000,34.469975,3.75,POLYGON ((139.76866170077992 35.69865290327819...
2,bldg_06682e6e-1fb7-46f2-a9f8-4c56c440e7d7,3.232244,3.010000,34.469975,3.75,POLYGON ((139.76869419869655 35.69865365084249...
3,bldg_a7b89248-7d33-4831-b334-e331e4e9eb6b,21.919524,4.610000,34.469975,3.75,POLYGON ((139.76962133641123 35.69807505155513...
4,bldg_46eea36d-a677-4701-b520-d08e5eb01613,14.520292,3.408000,34.469975,3.75,"POLYGON ((139.7692167578925 35.69782899732015,..."
...,...,...,...,...,...,...
95,bldg_09a48a56-b5c1-44ca-bc9f-db8ea2505f2f,12.919573,3.000000,37.942020,6.57,POLYGON ((139.76832693560007 35.70132204561928...
96,bldg_967943bc-f1ec-42ca-b2a7-3015517dddf7,35.219463,3.000000,37.942020,6.57,POLYGON ((139.76801925874472 35.70190100837792...
97,bldg_72da051d-215b-4ebb-b03f-dd6817e14496,81.685179,12.550000,37.942020,6.57,"POLYGON ((139.7687512320483 35.70165046232456,..."
98,bldg_f7822648-cbaf-4d22-83d5-989126ec2f38,230.984709,24.270000,37.942020,6.57,POLYGON ((139.76919208298065 35.70159607511183...


## クエリの結果をGeoDataFrameに格納

In [152]:
# change to GDF
df["geom"] = gpd.GeoSeries.from_wkt(df["geom"])
gdf = gpd.GeoDataFrame(df, geometry="geom")
gdf

,gml_id,cal_area_m2,cal_height_m,slope_max,elev_median,geom
0,bldg_3ec997d2-a7db-4729-aeed-56515ba1ab03,132.287307,3.060044,29.956266,3.11,"POLYGON ((139.75860 35.68325, 139.75849 35.683..."
1,bldg_c1bf1eb1-fb93-47ff-8d9d-0aa9ebd5f9f0,19.242557,7.710000,34.469975,3.75,"POLYGON ((139.76866 35.69865, 139.76869 35.698..."
2,bldg_06682e6e-1fb7-46f2-a9f8-4c56c440e7d7,3.232244,3.010000,34.469975,3.75,"POLYGON ((139.76869 35.69865, 139.76868 35.698..."
3,bldg_a7b89248-7d33-4831-b334-e331e4e9eb6b,21.919524,4.610000,34.469975,3.75,"POLYGON ((139.76962 35.69808, 139.76969 35.698..."
4,bldg_46eea36d-a677-4701-b520-d08e5eb01613,14.520292,3.408000,34.469975,3.75,"POLYGON ((139.76922 35.69783, 139.76925 35.697..."
...,...,...,...,...,...,...
95,bldg_09a48a56-b5c1-44ca-bc9f-db8ea2505f2f,12.919573,3.000000,37.942020,6.57,"POLYGON ((139.76833 35.70132, 139.76828 35.701..."
96,bldg_967943bc-f1ec-42ca-b2a7-3015517dddf7,35.219463,3.000000,37.942020,6.57,"POLYGON ((139.76802 35.70190, 139.76800 35.701..."
97,bldg_72da051d-215b-4ebb-b03f-dd6817e14496,81.685179,12.550000,37.942020,6.57,"POLYGON ((139.76875 35.70165, 139.76865 35.701..."
98,bldg_f7822648-cbaf-4d22-83d5-989126ec2f38,230.984709,24.270000,37.942020,6.57,"POLYGON ((139.76919 35.70160, 139.76905 35.701..."


## 地図表示

### leafmapを利用して、データを2D地図表示

In [153]:
# visualize data with 2D leafmap
m = leafmap.Map(center=(35.5744826,139.7078659), zoom=13)
m.add_gdf(gdf, layer_name="Plateau Buildings")
m

Map(center=[35.5744826, 139.7078659], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_titl…

### leafmap/deck.gl を利用して、データを3D地図表示

In [154]:
# visualize data in 3D with deck.gl
import leafmap.deck as leafmap

initial_view_state = {
    "latitude": 35.5744826,
    "longitude": 139.7078659,
    "zoom": 13,
    "pitch": 45,
    "bearing": 10,
}

m = leafmap.Map(initial_view_state=initial_view_state)

m.add_gdf(
    gdf,
    extruded=True,
    get_elevation="cal_height_m",
    elevation_range=[0, 3000],
    elevation_scale=3,
)
m

<IPython.core.display.Javascript object>

{
  "initialViewState": {
    "bearing": 10,
    "latitude": 35.5744826,
    "longitude": 139.7078659,
    "pitch": 45,
    "zoom": 13
  },
  "layers": [
    {
      "@@type": "GeoJsonLayer",
      "data": [
        {
          "cal_area_m2": 132.28730667374006,
          "cal_height_m": 3.0600440000000004,
          "elev_median": 3.109999895095825,
          "geometry": {
            "coordinates": [
              [
                [
                  139.75860220619106,
                  35.68324645774352
                ],
                [
                  139.75848993356308,
                  35.68327279818024
                ],
                [
                  139.75845180861046,
                  35.68316442964568
                ],
                [
                  139.758564081099,
                  35.6831380892439
                ],
                [
                  139.75860220619106,
                  35.68324645774352
                ]
              ]
            ],
            "type": "Polygon"
          },
          "gml_id": "bldg_3ec997d2-a7db-4729-aeed-56515ba1ab03",
          "slope_max": 29.956266403198242
        },
        {
          "cal_area_m2": 19.24255708046654,
          "cal_height_m": 7.71,
          "elev_median": 3.75,
          "geometry": {
            "coordinates": [
              [
                [
                  139.76866170077992,
                  35.698652903278195
                ],
                [
                  139.76869041298087,
                  35.698689162749275
                ],
                [
                  139.76864134692153,
                  35.698693282687316
                ],
                [
                  139.76862530816592,
                  35.698672073867115
                ],
                [
                  139.76861960133746,
                  35.69866451732673
                ],
                [
                  139.76865442614496,
                  35.69864371441923
                ],
                [
                  139.76866170077992,
                  35.698652903278195
                ]
              ]
            ],
            "type": "Polygon"
          },
          "gml_id": "bldg_c1bf1eb1-fb93-47ff-8d9d-0aa9ebd5f9f0",
          "slope_max": 34.469974517822266
        },
        {
          "cal_area_m2": 3.232243699776628,
          "cal_height_m": 3.01,
          "elev_median": 3.75,
          "geometry": {
            "coordinates": [
              [
                [
                  139.76869419869655,
                  35.698653650842495
                ],
                [
                  139.76868037930876,
                  35.69866193601595
                ],
                [
                  139.7686667998428,
                  35.6986467857219
                ],
                [
                  139.76868061922934,
                  35.69863850054997
                ],
                [
                  139.76869419869655,
                  35.698653650842495
                ]
              ]
            ],
            "type": "Polygon"
          },
          "gml_id": "bldg_06682e6e-1fb7-46f2-a9f8-4c56c440e7d7",
          "slope_max": 34.469974517822266
        },
        {
          "cal_area_m2": 21.91952424741113,
          "cal_height_m": 4.61,
          "elev_median": 3.75,
          "geometry": {
            "coordinates": [
              [
                [
                  139.76962133641123,
                  35.698075051555136
                ],
                [
                  139.76968841114024,
                  35.698032398196084
                ],
                [
                  139.76970744134718,
                  35.69805772771512
                ],
                [
                  139.76963541038313,
                  35.69809376241268
                ],
                [
                  139.76962133641123,
                  35.698075051555136
                ]
 